# Catálogo INE

> Datos que mantiene el Instituto Nacional de Estadística de Bolivia y cuándo los actualiza.

In [1]:
import pandas as pd
import datetime as dt
import pytz
from IPython.display import display, Markdown, HTML
import locale
from itables import show, init_notebook_mode

pd.options.display.max_colwidth=200
locale.setlocale(locale.LC_TIME, "es_US.UTF8")
init_notebook_mode()

<IPython.core.display.Javascript object>

- [Todos los datos](#Todos-los-datos)
- [Últimos cambios](#Últimos-cambios)

In [2]:
def draw_table(df):
    """
    Crea una tabla para explorar datos en el catálogo
    """
    colors = {
        'spreadsheet': '#f3f7ed',
        'excel': '#e1ede4',
        'pdf': '#f2e8e6',
        'rar': '#f2f0e6',
        'zip': '#f2f0e6',
        'stream': '#f7f6f5',
        'word': '#edf2f7'
    }

    dfi = df[['pagetitle', 'pagelink', 'modificado', 'nombre', 'tipo', 'kb', 'link']].rename(columns={'modificado': 'Modificado', 'nombre': 'Dato', 'tipo': 'Tipo', 'kb': 'Peso (KB)', 'link': '🔗'})
    dfi.Modificado = dfi.Modificado.dt.strftime('%Y-%m-%d %H:%M')
    dfi = dfi.set_index('Modificado')
    dfi.insert(0, 'Página', dfi[['pagetitle', 'pagelink']].apply(lambda row: '<a style="color: #919191" href="{}"><b>{}</b></a>'.format(row[1], row[0]), axis=1))
    dfi.drop(columns=['pagetitle', 'pagelink'])
    dfi['Dato'] = dfi[['🔗', 'Dato']].apply(lambda row: '<a style="color: #4f4f4f" href="{}"><b>{}</b></a>'.format(row[0], row[1]), axis=1)
    dfi['Tipo'] = dfi['Tipo'].apply(lambda x: '<span style="font-family:monospace; font-size:8px; border:1px solid #d7d6d6; border-radius:3px; opacity:.9; padding:3px; background:{}; color:"#000">{}</span>'.format(colors[x], x))
    dfi['Peso (KB)'] = dfi['Peso (KB)'].fillna(0).astype(int).apply(lambda x: '<span style="font-family:monospace; font-size:8px">{}</span>'.format(x))
    dfi = dfi[['Página', 'Dato', 'Tipo', 'Peso (KB)']]
    show(dfi, 
         order = [],
         hover = True,
         compact=True,
         scrollY="900px", 
         lengthMenu=[50,100],
         scrollCollapse=True,
         search={"caseInsensitive": True},
         paging=True,
         language={
             'lengthMenu': 'Mostrar _MENU_ filas',
             'search': '&#x1F50E;&#xFE0E;', 
             'processing': 'creando tabla ...', 
             'info': '', 
             'infoEmpty': '', 
             'infoFiltered':'_TOTAL_ documentos',
             'paginate': {
                'first': "Primero",
                'previous': "Anterior",
                'next': "Siguiente",
                'last': "Último"
            },
         }, 
         columnDefs=[
             {"width": "20px", "targets": [0, 1, 3]}, 
             {"width": "100px", "targets": 2}, 
             {'width': '5px', 'targets': [4]}
         ])

def status_color(s):
    return ['background:{}'.format("#CCFFD8" if s.Status == '+' else '#FFD7D5')] * 6

def mostrar_cambios():
    df_diff = pd.read_csv('catalogo_diff.csv', parse_dates=['modificado'])
    df_diff['nombre'] = df_diff[['nombre', 'link']].apply(lambda row: '<a style="color: #4f4f4f" href={}><b>{}</b></a>'.format(row[1], row[0]), axis=1)
    df_diff['pagetitle'] = df_diff[['pagetitle', 'pagelink']].apply(lambda row: '<a style="color: #4f4f4f" href={}><b>{}</b></a>'.format(row[1], row[0]), axis=1)
    df_diff['status'] = df_diff['status'].apply(lambda x: '<b>{}</b>'.format(x))
    df_diff = df_diff[['status', 'modificado', 'pagetitle', 'nombre', 'tipo', 'kb']]
    df_diff['kb'] = df_diff['kb'].fillna(0).astype(int)
    df_diff.columns = ['Status', 'Modificado', 'Página', 'Dato', 'Tipo', 'Peso (KB)']
    return df_diff.style.hide_index()
    
with open('update_time', 'r') as f:
    update_time = dt.datetime.strptime(f.read(), '%Y-%m-%d %H:%M')
display(Markdown("Datos actualizados {}.".format(update_time.strftime('**a las %H:%M del %-d de %B de %Y**'))))

display(Markdown('## Todos los datos'))
catdf = pd.read_csv('catalogo_ine.csv', parse_dates=['modificado']).sort_values('modificado', ascending=False)
draw_table(catdf)

display(Markdown('## Últimos cambios'))
display(mostrar_cambios())

Datos actualizados **a las 19:56 del 31 de julio de 2022**.

## Todos los datos

Loading... (need help?)


## Últimos cambios

Status,Modificado,Página,Dato,Tipo,Peso (KB)
-,2013-11-14 15:42:20-04:00,Boletas Censales,Censo 1976,pdf,107
+,2013-11-14 15:42:20-04:00,Cuestionarios Censales,Censo 1976,pdf,107
-,2021-04-29 12:02:56-04:00,Boletas Censales,Censo 1992,pdf,1688
+,2021-04-29 12:02:56-04:00,Cuestionarios Censales,Censo 1992,pdf,1688
-,2021-09-17 13:57:14-04:00,Longitud de Caminos,Bolivia – Longitud de Caminos según Dpto y Red 2000 – 2020,spreadsheet,190
-,2021-09-17 13:59:35-04:00,Longitud de Caminos,Bolivia – Longitud de Caminos según Red y Rodadura 2000 – 2020,spreadsheet,156
-,2021-09-24 12:09:10-04:00,Boletas Censales,Censo 1950,pdf,3643
-,2021-09-27 12:00:27-04:00,Boletas Censales,Censo 2012,pdf,875
+,2021-09-24 12:09:10-04:00,Cuestionarios Censales,Censo 1950,pdf,3643
+,2021-09-27 12:00:27-04:00,Cuestionarios Censales,Censo 2012,pdf,875


- [Historial de cambios](https://github.com/mauforonda/catalogo-ine/commits/master/catalogo_ine.csv)
- [Un listado de todos los documentos encontrados, incluyendo aquellos que dejaron de ser referenciados en ine.gob.bo](https://github.com/mauforonda/catalogo-ine/blob/master/catalogo_ine_historial.csv)
- [Otra vista de los datos](https://flatgithub.com/mauforonda/catalogo-ine?filename=catalogo_ine.csv&filters=&sort=modificado%2Cdesc&stickyColumnName=modificado)